<a href="https://colab.research.google.com/github/GeneSUN/data-science-engineering/blob/main/trip_duration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Distributional (probabilistic) regression demo for "trip duration (days)"
Goal: estimate P(duration > 30 | x), including for a NEW destination market (never seen before)

Key idea:
- Train a model that predicts a full conditional distribution p(y|x), not just E[y|x]
- Then compute exceedance probability via the CDF: P(Y>30)=1-F(30)

This script uses NGBoost (Natural Gradient Boosting for probabilistic prediction),
which predicts distribution parameters (e.g., LogNormal mean/scale) directly. :contentReference[oaicite:0]{index=0}
"""

import numpy as np
import pandas as pd

# -----------------------------
# 1) Synthesize tabular trip data
# -----------------------------
rng = np.random.default_rng(42)

N = 25000
n_origins = 60
n_destinations_seen = 140
n_destinations_new = 10

# "Market embeddings" (latent factors) for origin/destination
# These are *not* one-hot IDs; think of them as market features (geo, infra, ops, etc.)
k = 4
origin_emb = rng.normal(size=(n_origins, k))
dest_emb_seen = rng.normal(size=(n_destinations_seen, k))
dest_emb_new  = rng.normal(loc=0.5, scale=1.2, size=(n_destinations_new, k))  # shifted to simulate new market differences

# Create rows
origin_id = rng.integers(0, n_origins, size=N)
dest_id   = rng.integers(0, n_destinations_seen, size=N)

# Trip-level features (distance, weight, customs flag, seasonality, carrier reliability, etc.)
distance_km = rng.gamma(shape=2.2, scale=500.0, size=N)          # long tail distances
weight_kg   = rng.lognormal(mean=2.0, sigma=0.6, size=N)         # skewed
is_customs  = rng.binomial(1, 0.18, size=N)                      # cross-border/customs
peak_season = rng.binomial(1, 0.25, size=N)                      # seasonal congestion
carrier_rel = np.clip(rng.normal(loc=0.0, scale=1.0, size=N), -2, 2)  # higher is better

# Market features via embeddings
O = origin_emb[origin_id]                # (N,k)
D = dest_emb_seen[dest_id]               # (N,k)

# A latent "difficulty" score; new markets later will shift this
difficulty = (
    0.0020 * distance_km
    + 0.15  * np.log1p(weight_kg)
    + 0.55  * is_customs
    + 0.25  * peak_season
    - 0.20  * carrier_rel
    + 0.30  * (O[:,0] - D[:,0])
    + 0.15  * (O[:,1] * D[:,1])
)

# Convert to positive duration days using log-normal data generating process
# y ~ LogNormal(mu, sigma) with heteroscedastic sigma depending on features
mu_true = 2.2 + 0.35 * np.tanh(difficulty)          # controls typical duration
sigma_true = 0.35 + 0.10 * (is_customs + peak_season) + 0.05 * (distance_km > 1500)

y_days = rng.lognormal(mean=mu_true, sigma=sigma_true)

# Assemble training dataframe (NO raw IDs needed for generalization; keep embeddings + numeric features)
X = pd.DataFrame({
    "distance_km": distance_km,
    "log_weight": np.log1p(weight_kg),
    "is_customs": is_customs,
    "peak_season": peak_season,
    "carrier_rel": carrier_rel,
})
for j in range(k):
    X[f"origin_f{j}"] = O[:, j]
    X[f"dest_f{j}"]   = D[:, j]

y = y_days

# Train/valid split
idx = rng.permutation(N)
train_idx, val_idx = idx[: int(0.8*N)], idx[int(0.8*N):]
X_train, y_train = X.iloc[train_idx], y[train_idx]
X_val,   y_val   = X.iloc[val_idx],   y[val_idx]

# -----------------------------
# 2) Fit a distributional regression model (NGBoost)
# -----------------------------
# NGBoost predicts parameters of a chosen distribution p(y|x) and trains with proper scoring rules
# like Negative Log Likelihood. :contentReference[oaicite:1]{index=1}
from ngboost import NGBRegressor
from ngboost.distns import LogNormal

ngb = NGBRegressor(
    Dist=LogNormal,          # good for positive, skewed durations
    n_estimators=800,
    learning_rate=0.03,
    minibatch_frac=0.7,
    verbose=False,
    random_state=42
)

ngb.fit(X_train, y_train)

# -----------------------------
# 3) Exceedance probability: P(Y > 30 | x)
# -----------------------------
THRESH = 30.0

# NGBoost returns a distribution object per row with methods like cdf/survival.
# survival(y) = P(Y > y). (If survival not present, use 1 - cdf.)
pred_dist_val = ngb.pred_dist(X_val)

# Robustly compute exceedance
try:
    p_exceed_val = pred_dist_val.survival(THRESH)
except Exception:
    p_exceed_val = 1.0 - pred_dist_val.cdf(THRESH)

print("Example exceedance probabilities (validation):")
print(p_exceed_val[:10])

# -----------------------------
# 4) "New market" inference: destination never seen in training
# -----------------------------
# Key: you can only generalize to unseen destinations if you have destination FEATURES
# (geo, infrastructure, lead-time SLA, customs complexity, etc.), not only an ID.
# Here we simulate new destinations via new embeddings dest_emb_new.
M = 5  # make 5 hypothetical shipments to new markets
origin_id_new = rng.integers(0, n_origins, size=M)
dest_id_new   = rng.integers(0, n_destinations_new, size=M)

O2 = origin_emb[origin_id_new]
D2 = dest_emb_new[dest_id_new]

X_new = pd.DataFrame({
    "distance_km": rng.gamma(shape=2.2, scale=600.0, size=M),
    "log_weight": np.log1p(rng.lognormal(mean=2.1, sigma=0.7, size=M)),
    "is_customs": rng.binomial(1, 0.30, size=M),     # maybe higher for new markets
    "peak_season": rng.binomial(1, 0.25, size=M),
    "carrier_rel": np.clip(rng.normal(0.0, 1.0, size=M), -2, 2),
})
for j in range(k):
    X_new[f"origin_f{j}"] = O2[:, j]
    X_new[f"dest_f{j}"]   = D2[:, j]

pred_dist_new = ngb.pred_dist(X_new)
try:
    p_exceed_new = pred_dist_new.survival(THRESH)
except Exception:
    p_exceed_new = 1.0 - pred_dist_new.cdf(THRESH)

print("\nNew-market exceedance probabilities P(Y>30 | x):")
for i in range(M):
    print(f"trip{i}:  P(days>30)={float(p_exceed_new[i]):.3f}")

# -----------------------------
# 5) Notes for real-world "new market" (cold start)
# -----------------------------
# If your destination is truly unseen, avoid one-hot IDs.
# Use:
# - destination geo features, distance, lanes, customs, carrier network, port capacity, SLA tiers
# - embeddings learned from metadata (text, category, route graph)
# - hierarchical / partial pooling to borrow strength across markets (Bayesian / multilevel) :contentReference[oaicite:2]{index=2}
#
# For deep learning, you can do the same concept with torch.distributions + NLL: :contentReference[oaicite:3]{index=3}
